In [27]:
%matplotlib inline

In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as plt
import matplotlib.pyplot as plt

In [29]:
TEMPO = 120 #beats_per_second
BASE_FREQ = 256
PI = np.pi

In [30]:
DELTA_CENT_PA = np.floor(1200 * np.log(3/2)/np.log(2) + 0.5)
DELTA_CENT_GA = np.floor(1200 * np.log(5/4)/np.log(2) + 0.5)

In [31]:
m_list = [idx - 3 for idx in range(10)]
n_list = [idx - 1 for idx in reversed(range(5))]

In [32]:
samvad = []
for n in n_list:
    temp = []
    for m in m_list:
        temp += [(m * DELTA_CENT_GA + n * DELTA_CENT_PA) % 1200]
    samvad += [temp]
    
samvad_df = pd.DataFrame(samvad)
samvad_df.columns = [f'C_{idx + 1}' for idx in range(len(samvad_df.columns))]
samvad_df.index = [f'R_{idx + 1}' for idx in range(len(samvad_df.index))]

In [33]:
notes_df = pd.DataFrame({'cents' : samvad_df.stack()})
notes_df.index.names = ['rows', 'columns']
notes_df.reset_index(inplace = True)
notes_df.sort_values(by = 'cents', inplace = True)
notes_df.eval('N = floor(cents/100)', inplace = True)

In [34]:
octaves = []
for n in range(12):
    if n in [6, 11]:
        octaves += [notes_df.query('N == @n').iloc[[0]][['cents']]]
        continue
    
    octaves += [notes_df.query('N == @n').iloc[[0,-1]][['cents']]]
    
octaves_df = pd.concat(octaves, ignore_index = True)

In [35]:
octaves_df['notes'] = [
    'S',
    'r1', 'r2',
    'R1', 'R2',
    'g1', 'g2',
    'G1', 'G2',
    'M1', 'M2',
    'm1', 'm2',
    'P',
    'd1', 'd2',
    'D1', 'D2',
    'n1', 'n2',
    'N1', 'N2',
]

In [36]:
cents_df = octaves_df.set_index('cents')

In [37]:
swar_samvad_df = (
    notes_df.sort_index()
            .merge(cents_df, how = 'left', left_on = 'cents', right_index = True)
            .pivot(index = 'rows', columns = 'columns', values = 'notes')
).loc[[f'R_{idx + 1}' for idx in range(5)], [f'C_{idx + 1}' for idx in range(10)]]

In [38]:
swar_samvad_df

columns,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10
rows,,,,,,,,,,
R_1,NaN,NaN,M2,D2,r1,NaN,NaN,NaN,NaN,NaN
R_2,NaN,NaN,n2,R2,m1,n1,R1,NaN,NaN,NaN
R_3,NaN,NaN,g2,P,N1,g1,NaN,NaN,NaN,m2
R_4,NaN,NaN,d2,S,G1,d1,NaN,NaN,NaN,N2
R_5,NaN,NaN,r2,M1,D1,NaN,NaN,NaN,NaN,G2


In [39]:
three_octaves_df = pd.concat([
    pd.DataFrame({
        'notes': 'l_' + octaves_df['notes'],
        'cents': -1200 + octaves_df['cents']
    }),
    pd.DataFrame({
        'notes': octaves_df['notes'],
        'cents': octaves_df['cents']
    }),
    pd.DataFrame({
        'notes': 'u_' + octaves_df['notes'],
        'cents': 1200 + octaves_df['cents']
    }),
], ignore_index = True)

In [40]:
three_octaves_df.eval('freq = @BASE_FREQ * 2 ** (cents/1200)', inplace = True)

In [41]:
beats_df = pd.DataFrame({'n': np.arange(44100 * 10)})

In [42]:
beats_df.eval('t = n/44100', inplace = True)
beats_df.eval('beats = @TEMPO * t / 60', inplace = True)

In [43]:
 timbre_df = pd.concat([
     (
         beats_df.query(f'beats < {(n + 1) / 2}')
                  .eval(f'A = 3 * (beats / {(n + 1) / 2}) * exp( -9 * (beats / {(n + 1) / 2})**2)')
                  .eval(f'half_beats = {n + 1}')
     ) for n in range(8)
 ], ignore_index = True)

In [44]:
for freq, notes in three_octaves_df[['freq', 'notes']].values:
    timbre_df.eval(f'{notes} = A * sin(2 * @PI * {freq} * t)', inplace = True)

In [45]:
timbre_df.eval('rest = 0', inplace = True)

In [46]:
compose_df = pd.read_csv('compose.csv')

In [47]:
track_df = pd.DataFrame({
    'y': pd.concat([
        timbre_df.query('half_beats == @b')[f'{n}'] 
        for _, (b, n) in compose_df[['half_beats', 'notes']].iterrows()
    ], ignore_index = True)
})

In [48]:
track_df = (track_df - track_df.min())/(track_df.max() - track_df.min())

In [49]:
track_df = ((2**16 - 1) * track_df + 0.5).apply(np.floor)

In [50]:
with open('music.raw', 'wb') as wf:
    wf.write(bytearray(np.uint16(track_df.values)))

In [51]:
!rm music.mp3 

In [52]:
!ffmpeg -f u16le -ar 44100 -ac 1 -i music.raw music.mp3

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib